Using energy density, $\varepsilon$ where $m_r(r)$ is the mass enclosed in a radius, $\rho(r)$ is the density at some radius r, and
$P(r)$ is the pressure at radius $r$.

$$
\frac{dm_r}{dr} = 4\pi r^2 \varepsilon(r)
$$

$$
\frac{dP}{dr}=
    -G\frac{m_r(r)\varepsilon(r)}{r^2}
    \left(1+\frac{P(r)}{\varepsilon(r)}\right)
    \left(1+\frac{4\pi r^3 P(r)}{m_r(r)}\right)
    \left(1-\frac{2 Gm_r(r)}{r}\right)^{-1}
$$

$r$ = radius.

$m_r(r)$ = the radius enclosed within a radius $r$.

$P(r)$ = the pressure at radius $r$.

$\varepsilon$ = energy density at radius $r$.

$G$ = gravitational constant.



I am using a polytropic equation of state, which is defined as

$$
P(\rho) = K\rho^\frac{n+1}{n} = K\rho^{1 + \frac{1}{n}}
$$

For now, I'm just going to use the EOS

$$
P(\varepsilon) = K\varepsilon^\gamma 
$$

This equation can also be expressed as 

$$
\varepsilon(P) = \left(\frac{P}{K}\right)^\frac{1}{\gamma}
$$


In [ ]:
import numpy as np
from scipy.constants import G, pi
from scipy.integrate import solve_ivp

def energy_density(p):
    '''
    Calculates the energy density as a function of pressure.
    '''
    K = ...
    gamma = ...
    return (p / K) ** (1 / gamma)

def tov_rhs(r, state):
    '''
    Returns [dP/dr, dm/dr].
    '''
    # Pressure and mass
    p_r, m_r = state
    # Mass 
    dm_dr = 4*pi * r**2 * energy_density(r)
    # Pressure (split into factors)
    f1 = -G * ((m_r * energy_density(r)) / (r**2))
    f2 = 1 + (p_r / energy_density(r))
    f3 = 1 + ((4*pi * r**3 * p_r) / m_r)
    f4 = (1 - ((2*G*m_r) / r)) ** -1
    dp_dr = f1 * f2 * f3 * f4
    return [dp_dr, dm_dr]

R_0 = 0
R_MAX = 100
R_SPAN = (R_0, R_MAX)
P_0 = 1e-6
M_0 = 0

def solve():
    intial_conditions = (P_0, M_0)
    return solve_ivp(
        tov_rhs,
        R_SPAN,
        intial_conditions
    )
    
solutions = solve()
r_solutions = solutions.t
p_solutions = solutions.y[0]
m_solutions = solutions.y[1]

print(f"Radius solutions: {r_solutions}")
print(f"Pressure solutions: {p_solutions}")
print(f"Mass Solutions: {m_solutions}")

6.6743e-11
